# Dreadnode Artifact Logging

This notebook demonstrates how to log artifacts (files and directories) to your Dreadnode platform projects. Artifacts are preserved with their original structure and can be used to track assets.

### Benefits of Artifact Logging
Artifact logging provides several key benefits for your workflow:

- **Reproducibility**: Ensure your runs can reference the exact files used.
- **Organization**: Maintain the structure of complex file hierarchies.
- **Efficiency**: Content-based deduplication saves storage space.
- **Traceability**: Connect inputs, outputs, and code in one place.


In [ ]:
from dreadnode import (
    configure, run, log_artifact
)

configure(
    server="<YOUR DREADNODE PLATFORM URI>", # Replace with your server
    token="<YOUR API KEY>", # Replace with your token
    send_to_logfire=True,
    project="log-artifact"
)

Logfire project URL: ]8;id=725195;https://logfire-us.pydantic.dev/raja/starter-project\https://logfire-us.pydantic.dev/raja/starter-project]8;;\


## Logging Entire Directory as an Artifact
When you log a directory, Dreadnode SDK will preserve the entire directory structure and all files within it. This could be useful for keeping track of datasets, model checkpoints, or collections of related files in a project.

In [3]:
# Path to the directory we want to log
dir_path = "../../../data" # Replace with your directory path

# Log the directory as an artifact
with run() as r:
    log_artifact(dir_path)

05:11:24.228 wise-buzzard-624


## Logging a Single File as an Artifact
For individual files like pkl, image, or standalone models, you can log them directly:

In [4]:
# Path to an individual file
file_path = "../../../data/model.pkl" # Replace with your file path

# Log the file as an artifact
with run() as r:
    artifact = log_artifact(file_path)
    

05:21:07.327 versatile-moth-829


### Example of logging multiple artifacts in the same run

Dreadnode's artifact logging intelligently handles overlapping directories. This is particularly useful when logging multiple related directories in the same run.

Let's consider a file structure like this:
```bash
data/
└── audio/
    ├── subaudio/
    │   ├── file_example_MP3_2MG.mp3
    │   └── file_example_WAV_2MG.wav
    ├── subaudio2/
    │   └── file_example_OOG_2MG.ogg
    └── copied/
        ├── subaudio/
        │   ├── file_example_MP3_2MG.mp3
        │   └── file_example_WAV_2MG.wav
        └── subaudio2/
            └── file_example_OOG_2MG.ogg
```
When logging these directories, Dreadnode intelligently merges and deduplicates:

In [2]:
import json

# Define our paths
file_path_subaudio2 = "../../../data/audio/subaudio2"
file_path_subaudio = "../../../data/audio/subaudio"
file_path_audio = "../../../data/audio"
file_path_copied = "../../../data/audio/copied"

# Log in different orders to see the intelligent merging
with run("Smart Directory Merging") as r:
    # First log sub-directories, then parent directory
    artifact1 = log_artifact(file_path_subaudio2)
    artifact2 = log_artifact(file_path_subaudio)
    artifact3 = log_artifact(file_path_audio)    # Will merge previous two artifacts
    artifact4 = log_artifact(file_path_copied)   # Will be merged as a subdirectory
    
    # Get the final artifact trees
    print(f"Number of root artifacts: {len(r._artifacts)}")
    
    # Print first level of directories
    for artifact in r._artifacts:
        children = [child["dir_path"].split("/")[-1] if child["type"] == "dir" 
                  else child["final_real_path"].split("/")[-1] 
                  for child in artifact["children"]]
        print(f"Root directory: {artifact['dir_path']}")
        print(f"Children: {', '.join(children)}")

05:32:48.543 Smart Directory Merging
Number of root artifacts: 1
Root directory: /Users/raja/Desktop/dreadnode/data/audio
Children: .DS_Store, subaudio, copied, subaudio2


### Examining the Order-Dependent Scenario
Let's look more closely at our example of logging subdirectories before parent directories:

In [6]:
# Example 3: Mixed independent directories
with run("Mixed Directories") as r:
    log_artifact(file_path_subaudio)  # Independent subdirectory
    log_artifact(file_path_copied)    # Independent different subdirectory
    
    # Result: Two separate trees, no merging
    

05:23:28.303 Mixed Directories
